In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display, Markdown
import ipywidgets as widgets
import functools
from prim_algorithm import prim_algorithm

In [2]:
#config
color_standard = 'black'
color_new = 'gold'
color_old = 'blue'

font_size_node = 30
font_size_weight = 25
width_edge_not_visited = 5
width_edge_visited = 10

#Setting for visited nodes
visited = {
    'node_size': 3000,
    'node_color': 'cornflowerblue',
    'edgecolors': color_old,
    'linewidths': 5,
    }    
#Setting for last added node
last_visited = {
    'node_size': 3000,
    'node_color': 'white',
    'edgecolors': color_new,
    'linewidths': 5,
    }
#Setting for not visited nodes
not_visited = {
    'node_size': 3000,
    'node_color': 'white',
    'edgecolors': color_standard,
    'linewidths': 5,
    }

#adjust Setting
plt.rcParams['figure.figsize']=12,8
plt.rcParams['figure.dpi'] = 60

In [3]:
# observe/on_click Functions
def create_graph(change, Graph, display_plot, display_prim):
    # clear old inputs
    display_plot.clear_output()
    Graph.clear()

    # read and execute uploaded txt-file, get variables 
    uploaded_edges, pos, start_node = read_uploaded_variables(change)
    Graph.add_weighted_edges_from(uploaded_edges)

    # definiton slider
    temp_value = start_node + 1 if start_node != Graph.number_of_nodes else start_node - 1
    start_node_slider = widgets.IntSlider(value=temp_value, min=1, max=Graph.number_of_nodes(),
                                          description='Startnode:', continuous_update=False)

    # draw Graph on Plot
    labels = nx.get_edge_attributes(Graph, 'weight')
    nx.draw_networkx(Graph, pos, **not_visited, font_size=font_size_node, width=width_edge_not_visited)
    nx.draw_networkx_edge_labels(Graph, pos, edge_labels=labels, font_size=font_size_weight, rotate=False)

    # Display Plot and slider for start_node
    plt.axis('off')
    with display_plot:
        display(Markdown('## Original Network'))
        plt.show()
        display(start_node_slider)

    # on change of start_node, display the result of the prim_algoriithm step by step
    start_node_slider.observe( functools.partial(output_prim, Graph=Graph, display_prim=display_prim, 
                                                 pos=pos, labels=labels), names='value')
    start_node_slider.value = start_node


def output_prim(change, Graph, display_prim, pos, labels):
    # clear old output
    display_prim.clear_output()

    # variables
    start_node = change['new']
    used_edges = []  # edges of path that are colored

    # Prim algorithm
    path_edges, path_nodes, dfs_open_edges = prim_algorithm(Graph, start_node)

    # display every plot (+2 first and last Plot don´t have a df)
    for plot_nr in range(0, len(dfs_open_edges) + 2):
        display_text_and_df(display_prim, dfs_open_edges, plot_nr)
        draw_graph_on_plot(Graph, labels, path_nodes, plot_nr, pos, used_edges)

        plt.axis('off')
        with display_prim:
            plt.show()

        # update variable used_edges
        update_used_edges(dfs_open_edges, path_edges, plot_nr, used_edges)

    # output path length
    path_length = get_path_length(path_edges)
    with display_prim:
        display(Markdown('#### Length minimum spanning tree: ' + str(path_length) + ' Units'))

In [4]:
# line reducing functions 
def read_uploaded_variables(change):
    # read txt-Data and execute as code, get the some values
    for key in change['new']:
        string = change['new'][key]['content'].decode('utf-8')
        var_to_value = {}
        exec(string, var_to_value)

        # get variables
        start_node = var_to_value['start_node']
        uploaded_edges = var_to_value['edges']
        pos = var_to_value['pos']

    return uploaded_edges, pos, start_node


def display_text_and_df(display_prim, dfs_open_edges, plot_nr):
    # display text to plot
    if plot_nr == 0:
        with display_prim:
            display(Markdown('## Start Graph'))
    elif plot_nr == len(dfs_open_edges) + 1:
        with display_prim:
            display(Markdown('## Final Graph'))
    elif plot_nr != 0 and plot_nr != len(dfs_open_edges) + 1:
        with display_prim:
            display(Markdown('## ' + str(plot_nr) + '. Edge'))
    # display the possible edges as df except Startgraph, finished_Graph   
    if plot_nr != 0 and plot_nr != len(dfs_open_edges) + 1:
        with display_prim:
            display(dfs_open_edges[plot_nr - 1].style.hide_index())


def draw_graph_on_plot(Graph, labels, path_nodes, plot_nr, pos, used_edges):
    # Draw network components in Plot
    nx.draw_networkx_nodes(Graph, pos, nodelist=path_nodes[0:plot_nr], **visited)
    nx.draw_networkx_nodes(Graph, pos, nodelist=path_nodes[plot_nr:plot_nr + 1], **last_visited)
    nodelist_not_visited = list(set(list(Graph.nodes())) - set(path_nodes[:(plot_nr + 1)]))
    nx.draw_networkx_nodes(Graph, pos, nodelist=nodelist_not_visited, **not_visited)
    nx.draw_networkx_labels(Graph, pos, font_size=font_size_node)

    nx.draw_networkx_edges(Graph, pos, edgelist=(list(set(list(Graph.edges())) - set(used_edges))),
                           edge_color=color_standard, width=width_edge_not_visited)
    nx.draw_networkx_edges(Graph, pos, edgelist=used_edges[0:plot_nr], edge_color=color_old, width=width_edge_visited)
    nx.draw_networkx_edges(Graph, pos, edgelist=used_edges[plot_nr - 1:plot_nr], 
                           edge_color=color_new, width=width_edge_visited)
    nx.draw_networkx_edge_labels(Graph, pos, edge_labels=labels, font_size=font_size_weight, rotate=False)


def update_used_edges(dfs_open_edges, path, plot_nr, used_edges):
    # list ids
    start_edge_id = 0;
    end_edge_id = 1

    # update list for drawing used_edges in different colors
    if plot_nr != len(dfs_open_edges) and plot_nr != len(dfs_open_edges) + 1:
        # smaller number must be first when difference to Graph.edges() is needed (look draw_graph_on_plot)
        if path[plot_nr][start_edge_id] < path[plot_nr][end_edge_id]:
            used_edges.append(tuple((path[plot_nr][start_edge_id], path[plot_nr][end_edge_id])))
        elif path[plot_nr][end_edge_id] < path[plot_nr][start_edge_id]:
            used_edges.append(tuple((path[plot_nr][end_edge_id], path[plot_nr][start_edge_id])))


def get_path_length(path_edges):
    path_length = 0
    weight_id = 2
    # calculate path length
    for edge in path_edges:
        path_length = path_length + edge[weight_id]
    return path_length

In [5]:
#Create Widgets on start
display_plot = widgets.Output()
display_prim = widgets.Output()
uploader = widgets.FileUpload(accept='.txt', multiple=False)

#create empty Graph
G = nx.Graph()

#Displayed Widgets
display(Markdown('# Prim Algorithm')) 
display(Markdown('Upload your Graph:')) 
display(uploader)
display(display_plot)
display(display_prim)

#when Graph uploaded, use Prim Algorithm and Display result
uploader.observe(functools.partial(create_graph, Graph = G, display_plot = display_plot, 
                                   display_prim = display_prim), names = 'value')

# Prim Algorithm

Upload your Graph:

FileUpload(value={}, accept='.txt', description='Upload')

Output()

Output()